#Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [3]:
def entropy_func(class_count, num_samples):
    entropy = 0
    for count in class_count.values():
        probability = count / num_samples
        entropy -= probability * math.log2(probability)
    return entropy


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self):
        return self.group_classes.size

    def group_entropy(self):
        class_count = Counter(self.group_classes)
        return entropy_func(class_count, len(self.group_classes))


class Node:
    def __init__(self, split_feature, split_val, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data):
        if self.val is not None:
            return self.val
        if data[self.split_feature] <= self.split_val:
            return self.child_node_a.predict(data)
        else:
            return self.child_node_b.predict(data)


class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a, group_b):
        total_samples = len(group_a) + len(group_b)
        total_entropy = (len(group_a) / total_samples) * group_a.entropy + (len(group_b) / total_samples) * group_b.entropy
        return total_entropy

    def get_information_gain(self, parent_group, child_group_a, child_group_b):
        return parent_group.entropy - self.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_values, classes):
        best_gain = 0
        best_feature = None
        best_split_val = None
        parent_group = Group(classes)

        print(len(feature_values))
        print(range(len(feature_values[0])))
        for feature in range(len(feature_values[0])):
            chosen_feature = feature_values[:, feature]
            print('column')
            print(chosen_feature)
            print(len(chosen_feature))
            unique_values = np.unique(chosen_feature)
            print('uqnigeq')
            print(unique_values)
            for val in unique_values:
                temp_a_indexes = []
                temp_b_indexes = []
                for idx in range(len(chosen_feature)):
                    print(chosen_feature[idx])
                    if chosen_feature[idx] <= val:
                        temp_a_indexes.append(idx)
                    else:
                        temp_b_indexes.append(idx)
                group_a_indexes = np.asarray(temp_a_indexes, dtype=int)
                group_b_indexes = np.asarray(temp_b_indexes, dtype=int)
                print(val)
                print(group_a_indexes)
                print(group_b_indexes)
                group_a = Group(classes[group_a_indexes])
                group_b = Group(classes[group_b_indexes])
                print(group_a)
                print(group_b)

                gain = self.get_information_gain(parent_group, group_a, group_b)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_split_val = val

        return best_feature, best_split_val

    def get_best_split(self, data, classes):
        # zwrot najczęściej występującej klasy
        if self.depth >= self.max_depth or len(np.unique(classes)) == 1:
            return_class = Counter(classes).most_common(1)[0][0]
            print(return_class)
            return Node(val=Counter(classes).most_common(1)[0][0])

        best_feature, best_split_val = self.get_best_feature_split(data, classes)
        # if best_feature is None:
        #     return Node(val=Counter(classes).most_common(1)[0][0])

        temp_a_indexes = []
        temp_b_indexes = []
        for idx in range(len(data)):

            if data[idx, best_feature] <= best_split_val:
                temp_a_indexes.append(idx)
            else:
                temp_b_indexes.append(idx)
        group_a_indexes = np.asarray(temp_a_indexes, dtype=int)
        group_b_indexes = np.asarray(temp_b_indexes, dtype=int)

        self.depth += 1
        left_node = self.get_best_split(data[group_a_indexes], classes[group_a_indexes])
        right_node = self.get_best_split(data[group_b_indexes], classes[group_b_indexes])

        return Node(best_feature, best_split_val, self.depth, left_node, right_node)

    def build_tree(self, data, classes):
        self.tree = self.get_best_split(data, classes)

    def predict(self, data):
        return self.tree.predict(data)

In [14]:
# dc = DecisionTreeClassifier(3)
# dc.build_tree(x_train, y_train)
# for sample, gt in zip(x_test, y_test):
#     prediction = dc.predict(sample)
x_list = list(x_train)
class_count = Counter(x_list)
num = sum(class_count.values())
result = entropy_func(class_count, num)
print(result)

TypeError: unhashable type: 'numpy.ndarray'

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()

print("Features (data):")
print(iris.data)
print(len(iris.data))

print("\nTarget (labels):")
print(iris.target)

print("\nFeature Names:")
print(iris.feature_names)

print("\nTarget Names:")
print(iris.target_names)

Features (data):
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 